必需要的資料:

*熱門遊戲種類
*1."種類名稱" 2."遊戲名稱" 3."所屬公司" 4."發行日期" 5."評分" 6."遊戲介紹"

*遊戲新聞
*1."標題名稱" 2."時間" 3."地點" 4."新聞內容"

*展覽資訊
*1."主題名稱" 2."時間" 3."地點" 4."展覽內容"

In [8]:
def open_db():#開啟Database
    import pymysql.cursors
    db = pymysql.connect(host='usefordeepbowl.cukfiecfgfog.ap-northeast-1.rds.amazonaws.com',#連結MySQL
                             user='public_root',
                             password='thisisasamplepassword',                             
                             db='game_chatbot',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    return db

In [9]:
def search_db(table=None, columns = None, limit = None,operation = None, keyword = None, quantity=None):
    #talbe:資料表可能會有Join在一起的表，不過這是用字串儲存
    #columns:欄位的名稱
    #limit:篩選條件
    #keyword:要查的關鍵字,Ex:遊戲名等
    #quantity:目前無
    try:
        db = open_db()
        cursor = db.cursor()#建立資料庫游標
        sql = ("select "+columns+" \
                from "+table+" \
                where "+limit+" "+operation+" '"+keyword+"' ")#下指令
        
        cursor.execute(sql)
        rows = cursor.fetchall()
        return rows
    except Exception as e:
        print("Exeception occured:{}".format(e))
    finally:
        db.close()#關閉DataBase

In [34]:
def game_type(game_name = None):
    table = 'game inner join (game_has_type inner join type using (type_ID)) using (game_ID)'
    columns = 'distinct(game_name),type_name'
    game_name = '%'.join(i for i in game_name)
    ans = search_db(table,columns, limit = 'game_name',operation = 'like', keyword = game_name)#詢問遊戲價錢時有的columns:價錢、價錢單位
    
    if ans:
        ans = ans[0]['game_name'] + ',' + ','.join(i['type_name'] for i in ans)
    else:
        ans = 'haha none'
    return ans

In [62]:
def game_introduction(game_name = None):#遊戲介紹的Funtion\n",
    table = 'game'
    columns = 'game_introduction,game_release_date'#介紹遊戲時有的columns:介紹內容、發行日期
    ans = search_db(table,columns, limit = 'game_name', operation = 'like',keyword = game_name)
#     print(ans)
    if ans:
        ans = str(ans[0]['game_introduction']) + 'Release Date: ' + ans[0]['game_release_date'].strftime('%Y/%m/%d')
    else:
        ans = 'haha none',
    return ans

In [88]:
def game_news(game_name = None):#遊戲價錢的Funtion
    table = 'game'
    columns = 'game_price,game_price_currency'
    game_name = '%'.join(i for i in game_name)
    ans = search_db(table,columns, limit = 'game_name',operation = 'like',keyword = game_name)#詢問遊戲價錢時有的columns:價錢、價錢單位
    if ans:
        ans = str(ans[0]['game_price']) + ans[0]['game_price_currency']
    else:
        ans = 'haha none'
    return ans

In [2]:
import json
from flask import Flask, request, make_response, jsonify
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pymysql.cursors

app = Flask(__name__)


@app.route("/")
def verify():
    return "Hello world", 200


@app.route('/', methods=['POST'])#Dialogflow端
def webhook():
    req = request.get_json(silent=True, force=True)
    print(req)
    retJson = ''
#     for i in req['queryResult']['parameters']:
#         if i:
#             item = ''.join(i for i in res['queryResult']['parameters'])
#             print(item)
#         else:
#             print(i)
    if req['queryResult']['parameters']['game_introduction'] != None:
        keyword = req['queryResult']['parameters']['any']
        open_db()
        print(keyword)
        retJson = game_introduction(game_name = str(keyword))

    res = {"fulfillmentText": retJson}
    res = json.dumps(res,indent = 4)
    print(res)
    return res

# @app.route('/lastgame', methods=['GET'])#web端
# def webView():
#     cursor = db.cursor()#建立資料庫游標
    
#     sql = ("select * from game")#下指令
    
#     cursor.execute(sql)
#     rows = cursor.fetchall()
    
    
#     req = request.get_json(silent=True, force=True)
#     print(req)
    
#     return res

if __name__ == '__main__':
    app.run(port=5000)
    

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Dec/2018 18:42:17] "POST / HTTP/1.1" 200 -


{'responseId': 'a162e971-919c-4479-9be0-00a8a7da6d2b', 'queryResult': {'queryText': 'Mirror的遊戲介紹', 'action': 'action', 'parameters': {'game_type': '', 'any': 'Mirror', 'game_introduction': '介紹'}, 'allRequiredParamsPresent': True, 'fulfillmentMessages': [{'text': {'text': ['']}}], 'intent': {'name': 'projects/deepbowl-e7505/agent/intents/71659b1b-175b-465a-b508-067b18a7e475', 'displayName': 'GameChatBot'}, 'intentDetectionConfidence': 1.0, 'languageCode': 'zh-cn'}, 'originalDetectIntentRequest': {'payload': {}}, 'session': 'projects/deepbowl-e7505/agent/sessions/b7045468-a131-3321-8af9-585e4a41a25f'}
{'game_type': '', 'any': 'Mirror', 'game_introduction': '介紹'}
{'game_type': '', 'any': 'Mirror', 'game_introduction': '介紹'}
{'game_type': '', 'any': 'Mirror', 'game_introduction': '介紹'}
{
    "fulfillmentText": ""
}


['game_type', 'any', 'game_introduction']
